## 라이브러리 설치

In [ ]:
# 1. GPT-SoVITS 코드를 내 컴퓨터로 가져오기
git clone https://github.com/RVC-Boss/GPT-SoVITS.git
cd GPT-SoVITS

# 2. 필수 라이브러리 설치 (시간이 좀 걸립니다)
pip install -r requirements.txt

## 데이터 가공

In [ ]:
import os
import json
import glob

# [설정] 다운로드 받은 데이터가 있는 폴더 경로 (수정 필요)
# 예: C:/Downloads/AIHub_Novel/Training
BASE_DIR = "./Training"
OUTPUT_FILE = "filelists/train.txt"

def main():
    print("데이터 가공을 시작합니다...")

    # 결과 저장할 파일 열기
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        # 모든 JSON 파일 찾기 (재귀적 탐색)
        # 소설 데이터 폴더 구조에 맞춰 경로 패턴을 조정하세요.
        json_files = glob.glob(os.path.join(BASE_DIR, "**/*.json"), recursive=True)

        print(f"총 {len(json_files)}개의 JSON 파일을 찾았습니다.")

        for json_path in json_files:
            try:
                with open(json_path, "r", encoding="utf-8") as jf:
                    data = json.load(jf)

                # AI-Hub 문학작품 데이터 JSON 구조 파싱
                # 구조: root -> voice_piece -> tr (대본), filename (경로)
                if "voice_piece" in data:
                    vp = data["voice_piece"]
                    text = vp["tr"]  # 대본
                    wav_rel_path = vp["filename"] # JSON에 적힌 상대 경로

                    # JSON 파일 위치 기준으로 WAV 파일의 실제 절대 경로 찾기
                    # 보통 JSON과 WAV가 같은 폴더나 인접 폴더에 있습니다.
                    # 여기서는 JSON 파일명과 동일한 WAV가 있다고 가정하거나 경로를 조합합니다.

                    # (실제 데이터 구조에 따라 이 부분 수정 필요)
                    # 예: JSON이 /A/B/file.json 이고 WAV가 /A/B/file.wav 인 경우
                    wav_path = json_path.replace(".json", ".wav")

                    # WAV 파일이 실제로 존재하는지 확인
                    if os.path.exists(wav_path):
                        # 화자 이름 (임시로 폴더명이나 파일명에서 추출 추천)
                        # 여기서는 'Storyteller'로 통일하거나 JSON 내 메타정보 활용
                        speaker_name = "Narrator"

                        # GPT-SoVITS 포맷: 경로|대본|화자|언어
                        line = f"{wav_path}|{text}|{speaker_name}|Ko\n"
                        f.write(line)
            except Exception as e:
                print(f"Error parsing {json_path}: {e}")

    print("완료! filelists/train.txt 파일이 생성되었습니다.")

if __name__ == "__main__":
    main()